In [1]:
# saves you having to use print as all exposed variables are printed in the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import re
import pandas as pd
import numpy as np
import torch
import pyarrow as pa
import pyarrow.compute as pc
from nn_rag import Knowledge
from ds_core.handlers.abstract_handlers import ConnectorContract
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer, util


---------

In [3]:
from spacy.lang.en import English

In [5]:
text = """The familiar Poem

The fox jumps over the lazy dog.The lazy dog could be ball but anyway.
It had been a wild party.

The End"""

In [4]:
def paragraphs(document):
    start = 0
    for token in document:
        if token.is_space and token.text.count("\n") > 1:
            yield document[start:token.i]
            start = token.i
    yield document[start:]

In [6]:
nlp = English()
doc = nlp(text)

In [11]:
for p in paragraphs(doc):
    p = str(p).replace("\n", " ").strip()
    print(f'para - {p}')

para - The familiar Poem
para - The fox jumps over the lazy dog.The lazy dog could be ball but anyway. It had been a wild party.
para - The End
